In [1]:
#imports
from datasets import load_dataset
from thai2transformers.metrics import classification_metrics
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
#parameters
class Args:
    dataset_name_or_path = 'thainer'
    feature_col = 'tokens'
    label_col = 'pos_tags'
    metric_for_best_model = 'f1_macro'
    seed = 2020
    data_dir = '~/Downloads/LST20_Corpus'

args = Args()

In [3]:
if args.dataset_name_or_path == 'lst20':
    dataset = load_dataset(args.dataset_name_or_path,data_dir=args.data_dir)
else:
    dataset = load_dataset(args.dataset_name_or_path)
dataset

Reusing dataset thainer (/Users/admin/.cache/huggingface/datasets/thainer/thainer/1.3.0/e0a86672e5ad057c1093708597cdda3671a76e9b053d210a32205406726cca92)


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 6348
    })
})

In [4]:
if args.dataset_name_or_path == 'thainer' and args.label_col== 'ner_tags':
    dataset = dataset.map(lambda examples: {'ner_tags': [i if i not in [13,26] else 27 for i in examples[args.label_col]]})
    train_valtest_split = dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=args.seed)
    dataset['train'] = train_valtest_split['train']
    dataset['validation'] = train_valtest_split['test']
    val_test_split = dataset['validation'].train_test_split(test_size=0.5, shuffle=True, seed=args.seed)
    dataset['validation'] = val_test_split['train']
    dataset['test'] = val_test_split['test']
    tag_labels = dataset['train'].features[args.label_col].feature.names
    tag_labels = [tag_labels[i] for i in range(len(tag_labels)) if i not in [13,26]]
elif args.dataset_name_or_path == 'thainer' and args.label_col== 'pos_tags':
    train_valtest_split = dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=args.seed)
    dataset['train'] = train_valtest_split['train']
    dataset['validation'] = train_valtest_split['test']
    val_test_split = dataset['validation'].train_test_split(test_size=0.5, shuffle=True, seed=args.seed)
    dataset['validation'] = val_test_split['train']
    dataset['test'] = val_test_split['test']
    tag_labels = dataset['train'].features[args.label_col].feature.names
else:
    tag_labels = dataset['train'].features[args.label_col].feature.names
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 5078
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 635
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 635
    })
})

In [6]:
if args.dataset_name_or_path == 'thainer':
    from transformers import AutoTokenizer
    mbert_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    def pre_tokenize(token, space_token):
        token = token.replace(' ', space_token)
        return token
    def is_not_too_long(example,
                        max_length=510):
        tokens = sum([mbert_tokenizer.tokenize(
            pre_tokenize(token, space_token='<_>'))
                      for token in example[args.feature_col]], [])
        return len(tokens) < max_length
    dataset['test'] = dataset['test'].filter(is_not_too_long)
dataset['test']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
    num_rows: 621
})

In [7]:
%%time
#get sentence forms
def generate_sents(dataset, idx):
    features = dataset[idx][args.feature_col]
    labels = dataset[idx][args.label_col]
    return [(features[i], labels[i]) for i in range(len(features))]

train_sents = [generate_sents(dataset['train'],i) for i in range(len(dataset['train']))]
valid_sents = [generate_sents(dataset['validation'],i) for i in range(len(dataset['validation']))]
test_sents = [generate_sents(dataset['test'],i) for i in range(len(dataset['test']))]
len(train_sents), len(valid_sents), len(test_sents)

CPU times: user 1.21 s, sys: 33.3 ms, total: 1.24 s
Wall time: 1.26 s


(5078, 635, 621)

In [8]:
#generate x,y
def extract_features(doc, window=3, max_n_gram=3):
    #padding for words
    doc = ['xxpad' for i in range(window)] + doc + ['xxpad' for i in range(window)]            
    doc_features = []
    
    #for each word
    for i in range(window, len(doc)-window):
        #bias term
        word_features = ['bias'] 
        
        #ngram features
        for n_gram in range(1, min(max_n_gram+1,2+window*2)):
            for j in range(i-window,i+window+2-n_gram):
                feature_position = f'{n_gram}_{j-i}_{j-i+n_gram}'
                
                #word
                word_ = f'{"|".join(doc[j:(j+n_gram)])}'
                word_features += [f'word_{feature_position}={word_}']
        
        #append to feature per word
        doc_features.append(word_features)
    return doc_features

def generate_xy(all_tuples):
    #target
    y = [[str(l) for (w,l) in t] for t in all_tuples]
    #features
    x_pre = [[w for (w,l) in t] for t in all_tuples]
    x = [extract_features(x_, window=2, max_n_gram = 2) for x_ in tqdm(x_pre)]
    return x, y


x_train, y_train = generate_xy(train_sents)
if args.dataset_name_or_path=='lst20':
    import random
    random.seed(args.seed)
    x_train_small = random.sample(x_train,10000)
    random.seed(args.seed)
    y_train_small =  random.sample(y_train,10000)
else:
    x_train_small = x_train
    y_train_small = y_train
x_valid, y_valid = generate_xy(valid_sents)
x_test, y_test = generate_xy(test_sents)

In [9]:
import pycrfsuite
from sklearn.metrics import classification_report

def train_crf(model_name, c1, c2, x_train, y_train, max_iterations=500):
    # Train model
    trainer = pycrfsuite.Trainer(verbose=True)

    for xseq, yseq in tqdm(zip(x_train, y_train)):
        trainer.append(xseq, yseq)

    trainer.set_params({
        'c1': c1,
        'c2': c2,
        'max_iterations': max_iterations,
        'feature.possible_transitions': True,
        'feature.minfreq': 3.0,
    })

    trainer.train(f'{model_name}_{c1}_{c2}.model')
    
def evaluate_crf(model_path, features, labels, tag_labels):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_path)
    y_pred = []
    for xseq in tqdm(features, total=len(features)): y_pred.append(tagger.tag(xseq))
    preds = [int(tag) for row in y_pred for tag in row]
    labs = [int(tag) for row in labels for tag in row]
    return classification_report(labs,preds, target_names = tag_labels,digits=4),\
        f1_score(labs,preds,average='micro'),\
        f1_score(labs,preds,average='macro')

In [129]:
hyperparams = []
for c1 in tqdm([0.,0.5,1.]):
    for c2 in tqdm([0.,0.5,1.]):
        train_crf(args.dataset_name_or_path,c1,c2,x_train_small,y_train_small)
        report, f1_micro, f1_macro = evaluate_crf(f'{args.dataset_name_or_path}_{c1}_{c2}.model',
                                                  x_valid, y_valid, tag_labels)
        print(report)
        d = {'c1':c1, 'c2':c2, 'f1_micro':f1_micro, 'f1_macro': f1_macro, 'report':report}
        hyperparams.append(d)


Feature generation
type: CRF1d
feature.minfreq: 3.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 148711
Seconds required: 6.156

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 500
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 954751.579745
Feature norm: 1.000000
Error norm: 133359.655261
Active features: 148550
Line search trials: 1
Line search step: 0.000006
Seconds required for this iteration: 1.116

***** Iteration #2 *****
Loss: 818810.936978
Feature norm: 6.335856
Error norm: 185094.626285
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.709

***** Iteration #3 *****
Loss: 707805.971600
Feature norm: 6.235234
Error norm: 100584.678693
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #40 *****
Loss: 110339.957968
Feature norm: 140.095932
Error norm: 9048.291841
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.683

***** Iteration #41 *****
Loss: 107245.914148
Feature norm: 140.897737
Error norm: 3964.862757
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.664

***** Iteration #42 *****
Loss: 105267.391110
Feature norm: 139.635622
Error norm: 5094.316228
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.762

***** Iteration #43 *****
Loss: 103026.548544
Feature norm: 139.764259
Error norm: 4397.294429
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.816

***** Iteration #44 *****
Loss: 98753.318737
Feature norm: 143.202312
Error norm: 3470.789917
Active features: 148711
Line search trials: 1
Line search 

***** Iteration #79 *****
Loss: 52547.019556
Feature norm: 264.195633
Error norm: 4526.487048
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.587

***** Iteration #80 *****
Loss: 51985.204789
Feature norm: 263.864932
Error norm: 1699.600923
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.638

***** Iteration #81 *****
Loss: 51670.880523
Feature norm: 264.135788
Error norm: 979.774213
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.563

***** Iteration #82 *****
Loss: 51235.865903
Feature norm: 265.414936
Error norm: 1491.865646
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.600

***** Iteration #83 *****
Loss: 50583.252569
Feature norm: 267.868815
Error norm: 1832.598801
Active features: 148711
Line search trials: 1
Line search step:

***** Iteration #122 *****
Loss: 33388.232815
Feature norm: 358.544984
Error norm: 624.453284
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.596

***** Iteration #123 *****
Loss: 32942.520949
Feature norm: 361.931911
Error norm: 1057.679551
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.593

***** Iteration #124 *****
Loss: 32789.965845
Feature norm: 363.177128
Error norm: 1131.155124
Active features: 148711
Line search trials: 2
Line search step: 0.319703
Seconds required for this iteration: 1.115

***** Iteration #125 *****
Loss: 32603.930356
Feature norm: 363.651104
Error norm: 853.775621
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #126 *****
Loss: 32250.845073
Feature norm: 364.883783
Error norm: 614.775765
Active features: 148711
Line search trials: 1
Line search st

***** Iteration #162 *****
Loss: 25445.388002
Feature norm: 409.800830
Error norm: 405.105023
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.549

***** Iteration #163 *****
Loss: 25260.846724
Feature norm: 411.393084
Error norm: 598.034954
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.550

***** Iteration #164 *****
Loss: 25189.030945
Feature norm: 412.334357
Error norm: 560.551088
Active features: 148711
Line search trials: 2
Line search step: 0.431692
Seconds required for this iteration: 1.093

***** Iteration #165 *****
Loss: 25085.340215
Feature norm: 413.176424
Error norm: 393.891723
Active features: 148711
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.541

***** Iteration #166 *****
Loss: 24977.391926
Feature norm: 414.134056
Error norm: 400.909368
Active features: 148711
Line search trials: 1
Line search step

...

***** Iteration #85 *****
Loss: 93139.602233
Feature norm: 143.308045
Error norm: 301.181108
Active features: 32460
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.567

***** Iteration #86 *****
Loss: 93135.478821
Feature norm: 143.288108
Error norm: 149.826461
Active features: 32432
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #87 *****
Loss: 93132.572381
Feature norm: 143.279743
Error norm: 157.913591
Active features: 32426
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.579

***** Iteration #88 *****
Loss: 93129.599865
Feature norm: 143.264744
Error norm: 136.408999
Active features: 32412
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.576

***** Iteration #89 *****
Loss: 93126.619252
Feature norm: 143.248607
Error norm: 225.215448
Active features: 32370
Line search trials: 1
Line search step: 1.000000

***** Iteration #131 *****
Loss: 93082.175677
Feature norm: 143.107795
Error norm: 98.815281
Active features: 31977
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.556

***** Iteration #132 *****
Loss: 93081.876055
Feature norm: 143.104162
Error norm: 131.932654
Active features: 31975
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.559

***** Iteration #133 *****
Loss: 93081.340305
Feature norm: 143.104069
Error norm: 92.321866
Active features: 31965
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.557

***** Iteration #134 *****
Loss: 93081.054577
Feature norm: 143.101820
Error norm: 128.573584
Active features: 31958
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.571

***** Iteration #135 *****
Loss: 93080.534514
Feature norm: 143.101462
Error norm: 86.109003
Active features: 31962
Line search trials: 1
Line search step: 1.0000

***** Iteration #171 *****
Loss: 93070.824249
Feature norm: 143.089774
Error norm: 55.997995
Active features: 31898
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #172 *****
Loss: 93070.786308
Feature norm: 143.090103
Error norm: 91.142347
Active features: 31898
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.642

***** Iteration #173 *****
Loss: 93070.525400
Feature norm: 143.091613
Error norm: 46.649073
Active features: 31899
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.660

***** Iteration #174 *****
Loss: 93070.484982
Feature norm: 143.092258
Error norm: 88.211726
Active features: 31893
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.603

***** Iteration #175 *****
Loss: 93070.260850
Feature norm: 143.093526
Error norm: 47.402633
Active features: 31886
Line search trials: 1
Line search step: 1.000000

***** Iteration #211 *****
Loss: 93065.517209
Feature norm: 143.104994
Error norm: 68.959601
Active features: 31816
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.571

L-BFGS terminated with the stopping criteria
Total seconds required for training: 126.692

Storing the model
Number of active features: 31816 (148711)
Number of active attributes: 21457 (588372)
Number of active labels: 16 (16)
Writing labels
Writing attributes
Writing feature references for transitions
Writing feature references for attributes
Seconds required: 0.186




              precision    recall  f1-score   support

          NN     0.9345    0.9747    0.9542     70861
          VV     0.9394    0.9343    0.9368     47521
          PU     1.0000    0.9983    0.9991     45313
          CC     0.9279    0.9559    0.9417     18822
          PS     0.9404    0.9288    0.9346     13646
          AX     0.9300    0.9119    0.9208      7750
          AV     0.8934    0.7545    0.8181      7735
          FX     0.9959    0.9891    0.9925      8559
          NU     0.9744    0.8904    0.9305      7346
          AJ     0.8548    0.7817    0.8166      4979
          CL     0.8628    0.7662    0.8117      3940
          PR     0.8350    0.7958    0.8149      2238
          NG     1.0000    0.9978    0.9989      1795
          PA     0.9346    0.7967    0.8602       305
          XX     0.0000    0.0000    0.0000        45
          IJ     0.0000    0.0000    0.0000         5

    accuracy                         0.9466    240860
   macro avg     0.8139  

In [130]:
hyperparams_df = pd.DataFrame(hyperparams).sort_values('f1_macro',ascending=False).reset_index(drop=True)
best_hyperparams = hyperparams_df.iloc[0,:].to_dict()
hyperparams_df

,c1,c2,f1_micro,f1_macro,report
0,1.0,0.0,0.952271,0.803645,precision recall f1-score ...
1,0.5,0.0,0.951856,0.802050,precision recall f1-score ...
2,0.0,0.5,0.950801,0.801114,precision recall f1-score ...
3,0.5,0.5,0.950444,0.798502,precision recall f1-score ...
4,1.0,0.5,0.949195,0.797815,precision recall f1-score ...
5,0.5,1.0,0.948090,0.796299,precision recall f1-score ...
6,0.0,1.0,0.948829,0.796092,precision recall f1-score ...
7,1.0,1.0,0.946645,0.795668,precision recall f1-score ...
8,0.0,0.0,0.934692,0.790179,precision recall f1-score ...


In [131]:
print(best_hyperparams['report'])

              precision    recall  f1-score   support

          NN     0.9493    0.9769    0.9629     70861
          VV     0.9483    0.9451    0.9467     47521
          PU     1.0000    0.9991    0.9995     45313
          CC     0.9304    0.9583    0.9442     18822
          PS     0.9366    0.9329    0.9348     13646
          AX     0.9303    0.9160    0.9231      7750
          AV     0.8951    0.7620    0.8232      7735
          FX     0.9955    0.9891    0.9923      8559
          NU     0.9688    0.9055    0.9361      7346
          AJ     0.8564    0.8082    0.8316      4979
          CL     0.8594    0.7957    0.8263      3940
          PR     0.8114    0.8307    0.8209      2238
          NG     0.9994    0.9989    0.9992      1795
          PA     0.8953    0.8689    0.8819       305
          XX     0.0909    0.0222    0.0357        45
          IJ     0.0000    0.0000    0.0000         5

    accuracy                         0.9523    240860
   macro avg     0.8167   

In [20]:
#final model
c1, c2 = best_hyperparams['c1'], best_hyperparams['c2']
train_crf(f'{args.dataset_name_or_path}_{args.label_col}_best',c1,c2,x_train,y_train)

In [14]:
#debug
c1, c2 = 0.5, 0.0

if args.dataset_name_or_path=='lst20' and args.label_col=='ner_tags':
    report, f1_micro, f1_macro = evaluate_crf(f'{args.dataset_name_or_path}_{args.label_col}_best_{c1}_{c2}.model',
                                              x_test, y_test, tag_labels[:-1]) #test set of lst20 does not have E_TTL
    print(report)
else:
    report, f1_micro, f1_macro = evaluate_crf(f'{args.dataset_name_or_path}_{args.label_col}_best_{c1}_{c2}.model',
                                              x_test, y_test, tag_labels)
    print(report)


              precision    recall  f1-score   support

         ADJ     0.8787    0.7263    0.7953       369
         ADP     0.9487    0.9322    0.9404      1032
         ADV     0.8333    0.6818    0.7500       374
         AUX     0.9645    0.9685    0.9665       953
       CCONJ     0.9760    0.9613    0.9686       465
         DET     0.9302    0.8542    0.8906       343
        NOUN     0.9261    0.9711    0.9481      8410
         NUM     0.9753    0.9180    0.9458       732
        PART     0.9923    0.9556    0.9736       270
        PRON     0.9452    0.9139    0.9293       151
       PROPN     0.9085    0.8121    0.8576       660
       PUNCT     0.9974    0.9987    0.9981      3893
       SCONJ     0.9664    0.9485    0.9574      1243
        VERB     0.9280    0.9042    0.9159      3205

    accuracy                         0.9450     22100
   macro avg     0.9408    0.8962    0.9169     22100
weighted avg     0.9447    0.9450    0.9442     22100

